In [327]:
TEST_DIR = '../test/'
IN_SHAPE_DIR = '../test/in_shape/'

# Preprocess route data

In [358]:
import fiona
from fiona.crs import from_epsg
from shapely.geometry import Point, LineString, shape, mapping
import rtree
from shapely.ops import snap, transform
import copy
import pyproj
from functools import partial

In [359]:
def snap_linestrings(r, lines):
    # create an empty spatial index object
    index = rtree.index.Index()
    
    snapped = []
    
    # populate the spatial index
    for index_id, geom in enumerate(lines):
        index.insert(index_id, geom.bounds)
    
    # create snapped lines
    for search_id, geom in enumerate(lines):
        
        # buffer the line
        buffered = geom.buffer(r)

        # get list of fids where bounding boxes intersect
        index_ids = [int(i) for i in index.intersection(buffered.bounds)]

        # access the features that those fids reference
        new_geom = copy.deepcopy(geom)
        for index_id in index_ids:
            
            if search_id == index_id:
                continue
            
            new_geom = snap(new_geom, lines[index_id], r)
            
            index.delete(search_id, geom.bounds)
        
        snapped.append(new_geom)
    
    return snapped

In [360]:
def write_shape(geometries, data, schema, path):
    with fiona.open(path, 'w', crs=from_epsg(4326), driver='ESRI Shapefile', schema=schema) as output:
        for i, g in enumerate(geometries):
            # attributes
            attributes = data[i]
            # write the row (geometry + attributes in GeoJSON format)
            output.write({'geometry': mapping(g), 'properties': attributes})

In [442]:
def project_to_utm(geoms):
    project_to_utm = partial(
        pyproj.transform, 
        pyproj.Proj(init='epsg:4326'),
        pyproj.Proj(init="epsg:28355", proj="utm", zone=55)
    )
    return [transform(project_to_utm, geom) for geom in geoms]

def project_from_utm(geoms):
    project_from_utm = partial(
        pyproj.transform, 
        pyproj.Proj(init="epsg:28355", proj="utm", zone=55),
        pyproj.Proj(init='epsg:4326')
    )
    return [transform(project_from_utm, geom) for geom in geoms]

In [443]:
# Load road shapefile
ROAD_PATH = TEST_DIR + 'ne_10m_roads_test.shp'
roadF = fiona.open(ROAD_PATH)
road_geoms = [LineString(shape(f['geometry'])) for f in roadF]
road_data = [f['properties'] for f in roadF]

# Project to UTM coordinate system
road_geoms = project_to_utm(road_geoms)

# Create new snapped LineString collection
SNAPPED_FILE_NAME = 'snapped.shp'
snapped = snap_linestrings(0.005, road_geoms)

back = project_from_utm(snapped)
write_shape(back, road_data, roadF.schema, IN_SHAPE_DIR + SNAPPED_FILE_NAME)

# Preprocess livestock and crop data into points

In [364]:
import rasterio
import numpy as np
from affine import Affine
from collections import OrderedDict

In [365]:
# Load livestock raster
LIVESTOCK_PATH = TEST_DIR + 'csiro_bvmeat_feed_00_test.tif'

# Read raster
with rasterio.open(LIVESTOCK_PATH) as r:
    T0 = r.transform  # upper-left pixel corner affine transform
    p1 = Proj(r.crs)
    A = r.read()  # pixel values

# Get affine transform for pixel centres
T1 = T0 * Affine.translation(0.5, 0.5)
# Function to convert pixel row/column index (from 0) to lat/lon at centre
rc2en = lambda r, c: (c, r) * T1

# Potential way to vectorise:

# All rows and columns into numpy mesh grid
# cols, rows = np.meshgrid(np.arange(A.shape[2]), np.arange(A.shape[1]))

# All eastings and northings (there is probably a faster way to do this)
# lats, lons = np.vectorize(rc2en, otypes=[np.float, np.float])(rows, cols)

In [367]:
point_geoms = []
point_data = []
it = np.nditer(A, flags=['multi_index'])

# TODO this is pretty inefficient, could come up with a vectorised implementations like that in the cell above
while not it.finished:
    value = np.asscalar(it[0])
    if value > 0:
        point_geoms.append(Point(rc2en(it.multi_index[1], it.multi_index[2])))
        point_data.append(
            {'value': np.asscalar(it[0])}
        )
    it.iternext()

point_geoms = project_to_utm(point_geoms)

back = project_from_utm(point_geoms)
schema = OrderedDict({
    'geometry': 'Point',
    'properties': {'value': 'float:16'}
})
write_shape(back, point_data, schema, IN_SHAPE_DIR + 'point_test.shp')

# Assign livestock and crop data to route locations

In [429]:
from shapely.ops import nearest_points, split, snap
from shapely.geometry import MultiPoint

In [447]:
def make_points_on_line(geom, distance):
    if geom.geom_type == 'LineString':
        num_vert = int(round(geom.length / distance))
        if num_vert == 0:
            num_vert = 1
        return [
            geom.interpolate(float(n) / num_vert, normalized=True)
            for n in range(num_vert + 1)
        ]
    elif geom.geom_type == 'MultiLineString':
        parts = [make_points_on_line(part, distance)
                 for part in geom]
        return type(geom)([p for p in parts if not p.is_empty])
    else:
        raise ValueError('unhandled geometry %s', (geom.geom_type,))

        
def split_line_by_distance(geom, distance):
    m = MultiPoint(make_points_on_line(geom, distance))
    return split(geom, m)
    

def join_points_to_lines(points, lines, step):
    
    # create an empty spatial index object
    index = rtree.index.Index()
    
    snapped = []
    
    # populate the spatial index
    for index_id, geom in enumerate(lines):
        index.insert(index_id, geom.bounds)
    
    # create lines which join points to lines
    joins = []
    for search_id, geom in enumerate(points):
        
        # buffer out the point until we hit a line
        
        r = step
        index_ids = []
        
        while len(index_ids) == 0:  
            buffered = geom.buffer(r)
            index_ids = [int(i) for i in index.intersection(buffered.bounds)]
            r += step

        closest_line = lines[index_ids[0]]  # doesn't seem to work well
        
        # nearest_points method (attaches anywhere on a line)
        # join_points = nearest_points(closest_line, geom)
        # joins.append(LineString(join_points))
        
        # snap method (only attaches to line endpoints)
        closest_point_on_line = snap(geom, closest_line, r*100)
        joins.append(LineString((geom, closest_point_on_line)))
    
    return joins

In [375]:
joins = join_points_to_lines(point_geoms, snapped, 100)

In [376]:
back = project_from_utm(joins)
schema = OrderedDict({
    'geometry': 'LineString',
    'properties': {'value': 'float:16'}
})
write_shape(back, point_data, schema, IN_SHAPE_DIR + 'join_test.shp')

In [451]:
# lines = split_line_by_distance(snapped[0], 10000)
snap_back = project_from_utm(snapped)
points = make_points_on_line(snap_back[0], 0.1)
schema = OrderedDict({
    'geometry': 'LineString',
    'properties': {}
})
pointSchema = OrderedDict({
    'geometry': 'Point',
    'properties': {}
})
# back = project_from_utm(lines)
# pointBack = project_from_utm(points)

lines = split_line_by_distance(snap_back[0], 0.1)
print(lines)
print(points[0])

write_shape(lines, [{} for b in lines], schema, TEST_DIR + 'line_split_test.shp')
write_shape(points, [{} for b in points], pointSchema, TEST_DIR + 'points_on_line_test.shp')


GEOMETRYCOLLECTION (LINESTRING (148.2473281840724 -41.51390395148999, 148.2514526062891 -41.52215279592359, 148.2679502951563 -41.52215279592358, 148.2720747173731 -41.55927259587475, 148.2968212506739 -41.57989470695874, 148.3009456728907 -41.59226797360913, 148.3050700951075 -41.6170145069099, 148.2968212506739 -41.64176104021069, 148.2761991395899 -41.66650757351147, 148.2720747173731 -41.68300526237866, 148.2720747173731 -41.71600064011302, 148.2679502951563 -41.76961812893138, 148.2514526062891 -41.78199139558178, 148.2597014507227 -41.82736003996654, 148.2514526062891 -41.83973330661693, 148.2555770285059 -41.85623099548411, 148.2720747173731 -41.87685310656812, 148.2968212506739 -41.88510195100169, 148.2926968284571 -41.88922637321849, 148.2638258729395 -41.89335079543528, 148.2514526062891 -41.91397290651928, 148.2308304952052 -41.92634617316968, 148.2225816507716 -41.93871943982005, 148.193710695254 -41.95521712868723, 148.1689641619532 -41.95934155090403, 148.1442176286524 -4

# Convert to graph and run analysis

In [298]:
import networkx as nx

In [299]:
Groad = nx.read_shp(IN_SHAPE_DIR)

nx.write_shp(Groad, TEST_DIR)

In [300]:
Groad.nodes()

NodeView(((148.24732818407236, -41.51390395148999), (148.0534803398829, -42.05420326189036), (146.65117678617202, -41.52627721814038), (146.8161536748439, -42.02945672858958), (145.83454118724626, -41.047844240991964), (145.53758278763692, -42.058327684107155), (147.48843449618175, -41.93459501760326), (147.14198302997082, -41.47678415153882), (146.45732894198258, -41.249940929615), (147.1708539854884, -41.571645862525145), (147.443065851797, -41.839733306616935), (147.3069599186427, -42.89558539411689), (147.0512457412013, -43.03169132727118), (145.25712207689475, -40.812752174634554), (146.74191407494155, -41.09321288537673), (147.13373418553724, -41.44378877380445), (146.96875729686536, -43.43176028230046), (146.88626885252944, -41.11795941867751), (147.39769720741222, -41.01897328547439), (147.83901038460948, -41.10971057424391), (147.51730545169934, -41.159203640845476), (146.82440251927747, -41.060217507642356), (147.5585496738673, -42.78422599426338), (147.84313480682627, -43.14

In [301]:
try:
    n = nx.shortest_path_length(G,(148.24732818407236, -41.51390395148999),(146.9440107635646, -41.30355841843336))
    print(n)
except nx.NetworkXNoPath:
    print('No path')

No path
